# Stock Market Performance Analysis
[Reference](https://thecleverprogrammer.com/2023/05/08/stock-market-performance-analysis-using-python/)

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime
import plotly.express as px
pd.set_option('display.max_rows', None)

In [ ]:

start_date = datetime.now() - pd.DateOffset(months=3)
end_date = datetime.now()

tickers = ['AAPL', 'MSFT', 'NFLX', 'GOOG']

df_list = []

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    df_list.append(data)

# Add all df into 1 df, with keys as tickers
df = pd.concat(df_list, keys=tickers, names =['Ticker', 'Date'])
df = df.reset_index()

In [ ]:
df1 = df.copy()
df1.head(10)

In [ ]:
fig = px.line(df1, x='Date',
              y='Close',
              color='Ticker',
              title="Stock Market Performance for the Last 3 Months")
fig.show()

In [ ]:
fig = px.area(
    df1,
    x = "Date", y="Close",
    color = "Ticker",
    facet_col= "Ticker",
    labels = {"Date":"Date", "Close":"Closing Price", "Ticker":"Company"},
    title = "Stock price for Apple, Microsoft, Netflix, Google" 
)

fig.show()

In [ ]:
df1['MA10'] = df1.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df1['MA20'] = df1.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df1.groupby('Ticker'):
    print(f'Moving average for {ticker}')
    print(group[['MA10', 'MA20']])


In [ ]:
df2 = df1.groupby('Ticker')
df2.head(20)

In [ ]:
for ticker, group in df1.groupby('Ticker'):
    fig = px.line(
        group, 
        x='Date', y=['Close', 'MA10', 'MA20'],
        title = f"{ticker} Moving Average"
        )
    fig.show()

- When the **MA10 crosses above the MA20**, it is considered a bullish signal indicating that the stock price will continue to rise. 
    <br> -> Buy signal
- Conversely, when the **MA10 crosses below the MA20**, it is a bearish signal that the stock price will continue falling.
    <br> -> Sell signal

In [ ]:
df1["Volatility"] = df1.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)

fig = px.line(
    df1,
    x="Date", y="Volatility",
    color = "Ticker",
    title = "Volatility of all companies"
)

fig.show()

- High volatility indicates that the stock or market experiences large and frequent price movements, 
- While low volatility indicates that the market experiences smaller or less frequent price movements.

In [ ]:
# create a DataFrame with the stock prices of Apple and Microsoft
apple = df1.loc[df1['Ticker'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close':'AAPL'})
microsoft = df1.loc[df1['Ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close':'MSFT'})
apple

In [ ]:
def create_df(comp):
    df_comp = df1.loc[df1['Ticker'] == f'{comp}', ['Date', 'Close']].rename(columns={'Close':f'{comp}'})
    return df_comp
    
apple = create_df('AAPL')
microsoft = create_df('MSFT')

df_corr = pd.merge(apple, microsoft, on='Date')

# create a scatter plot to visualize correlation

fig = px.scatter(
    df_corr,
    x='AAPL', y = 'MSFT',
    trendline='ols',
    title = "Correlation between Apple and Microsoft"
    )

fig.show()


There is a strong linear relationship between the stock prices of Apple and Microsoft, which means that when the stock price of Apple increases, the stock price of Microsoft also tends to increase.